#  BackSlices Algorythm  
## Data preprocessing and features generation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from monthdelta import monthdelta, monthmod

from dateutil import relativedelta
pd.set_option('display.max_columns', 150)

In [2]:
import warnings
warnings.filterwarnings('ignore')

#### Transformation Parameters

In [3]:
## output file suffix
suffix = 'Transform_data'

In [4]:
## prediction window width, months:
window = 1
## prediction step, months: (time window offset)
step = 1
## buffer period, months:
buffer_period = 1
## back slices qty:
n_slices = 10
## minimum historу length, months
history_min = 5

### Load the data

In [5]:
data = pd.read_csv('data/data_long_prep_add.csv', sep=';', 
                   parse_dates=['Timeline', 
                                'DateofTermination'],
                  decimal='.')

In [6]:
data.shape

(308303, 98)

In [7]:
# DOT = data.DateofTermination
# print(np.min(DOT))
# print(np.max(DOT))

### Data preparation

In [8]:
data['Projects'] = data['#ofProjects']
data = data.drop('#ofProjects', axis=1)

In [9]:
## Numeric features list

number_cols = [

'Number of courses',
'Training days',
'English training',
'English training days',
'Projects',
'SickLeave',
'DurationOfSickLeaves',
'Vacation',
'DurationOfVacations',
'pages_loaded',
'likes',
'comments',
'posts',
    
]

In [10]:
data[number_cols].dtypes

Number of courses        float64
Training days            float64
English training         float64
English training days    float64
Projects                 float64
SickLeave                float64
DurationOfSickLeaves     float64
Vacation                 float64
DurationOfVacations      float64
pages_loaded             float64
likes                    float64
comments                 float64
posts                    float64
dtype: object

### Filters

In [11]:
# print(data.shape)
# data = data[data.History >= history_min]
# print(data.shape)

(308303, 98)
(267133, 98)


### New structure formation

In [12]:
tmp = data[['TL_number', 'Timeline', 'TerminatedOrResigned']].copy()
tmp.groupby(['TL_number', 'Timeline']).count()

,,TerminatedOrResigned
TL_number,Timeline,
0.0,2015-07-01,52
1.0,2015-08-01,56
2.0,2015-09-01,48
3.0,2015-10-01,64
4.0,2015-11-01,88
5.0,2015-12-01,88
6.0,2016-01-01,60
7.0,2016-02-01,83
8.0,2016-03-01,98


#### Remove a month with incomplete data

In [13]:
print(data.shape)
data = data[data.Timeline <= '2018-06-01']
print(data.shape)

(267133, 98)
(258289, 98)


In [14]:
Timelines = data.Timeline
Timelines = Timelines.drop_duplicates()
Timelines = sorted(Timelines, reverse=True)
TL = len(Timelines)  
TL

36

In [15]:
Timelines[0]

Timestamp('2018-06-01 00:00:00')

In [16]:
Timelines[35]

Timestamp('2015-07-01 00:00:00')

#### Missing values processing

In [17]:
for col in number_cols:
    if np.sum(data[col].isnull())>0: 
        print("'"+col+"',")

'Number of courses',
'Training days',
'English training',
'English training days',
'pages_loaded',
'likes',
'comments',
'posts',


In [18]:
nul_cols = [
'Number of courses',
'Training days',
'English training',
'English training days',
]

In [19]:
minus_one_cols = [
'pages_loaded',
'likes',
'comments',
'posts',
]

In [20]:
for col in nul_cols:
    data.loc[:,col] = data.loc[:,col].fillna(0)

for col in minus_one_cols:
    data.loc[:,col] = data.loc[:,col].fillna(-1)

In [21]:
data.shape

(258289, 98)

### Data transformation

In [22]:
number_cols.append('Timeline')
number_cols.append('PIN')
number_cols

['Number of courses',
 'Training days',
 'English training',
 'English training days',
 'Projects',
 'SickLeave',
 'DurationOfSickLeaves',
 'Vacation',
 'DurationOfVacations',
 'pages_loaded',
 'likes',
 'comments',
 'posts',
 'Timeline',
 'PIN']

In [23]:
tmp = data[number_cols].copy()          

In [24]:
%%time
melt_data = pd.melt(tmp, id_vars=['PIN','Timeline'], 
                       value_name='value')
melt_data['variables'] = melt_data.variable + '_' + melt_data.Timeline.astype(str)
print(melt_data.shape)

(3357757, 5)
Wall time: 38.2 s


In [25]:
melt_data.head()

,PIN,Timeline,variable,value,variables
0,25,2015-07-01,Number of courses,0.0,Number of courses_2015-07-01
1,137,2015-07-01,Number of courses,0.0,Number of courses_2015-07-01
2,213,2015-07-01,Number of courses,0.0,Number of courses_2015-07-01
3,233,2015-07-01,Number of courses,0.0,Number of courses_2015-07-01
4,343,2015-07-01,Number of courses,0.0,Number of courses_2015-07-01


In [26]:
%%time
cast_data = melt_data.pivot(index='PIN', columns='variables', values='value')
cast_data.reset_index(level=[0], inplace=True)
print(cast_data.shape)

(13688, 469)
Wall time: 2.69 s


In [27]:
cast_data.iloc[:,:4].head()

variables,PIN,DurationOfSickLeaves_2015-07-01,DurationOfSickLeaves_2015-08-01,DurationOfSickLeaves_2015-09-01
0,10,0.0,0.0,0.0
1,13,0.0,0.0,0.0
2,21,0.0,0.0,0.0
3,22,0.0,0.0,0.0
4,23,0.0,0.0,0.0


In [28]:
Timelines = np.unique(data.Timeline.astype(str))
Timelines = list(Timelines)
Timelines.reverse()
Timelines[:5]

['2018-06-01', '2018-05-01', '2018-04-01', '2018-03-01', '2018-02-01']

In [29]:
range(window-1, TL-(n_slices + buffer_period), step)

range(0, 25)

In [30]:
%%time
df = pd.DataFrame()
for i in range(window-1, TL-(n_slices + buffer_period), step):
   
    buffer_df = pd.DataFrame()
    tl0 = Timelines[i]
    print(tl0)
    
    for j in range(1,n_slices+1):
        tl = '_' + Timelines[i + j + buffer_period]
        suffix_ = '_' + str(j-1)
        names = [c for c in cast_data.columns if c.endswith(tl)]
        cast = cast_data[names]
        col  = map(lambda x: x.replace(tl,suffix_), names)   
        cast.columns = col
        buffer_df = pd.concat((buffer_df,cast), axis=1)    
    
    buffer_df.loc[:,'Timeline'] = tl0
    buffer_df.loc[:,'PIN'] = cast_data.PIN
    buffer_df = buffer_df.fillna(0)
    
    df = df.append(buffer_df, ignore_index=True)

2018-06-01
2018-05-01
2018-04-01
2018-03-01
2018-02-01
2018-01-01
2017-12-01
2017-11-01
2017-10-01
2017-09-01
2017-08-01
2017-07-01
2017-06-01
2017-05-01
2017-04-01
2017-03-01
2017-02-01
2017-01-01
2016-12-01
2016-11-01
2016-10-01
2016-09-01
2016-08-01
2016-07-01
2016-06-01
Wall time: 8.66 s


In [31]:
number_cols.remove('PIN')
number_cols.remove('Timeline')
tmp = data.drop(number_cols, axis=1)

In [32]:
selector = [
'PIN',
'Timeline',    
'DateofTermination',
'City',
'Country',
'Specialization',
'Sex',
'TerminatedOrResigned',
'MonthsInCurrentPosition',
'TL_number',
]
tmp = tmp[selector]

#### Target creation

In [33]:
index = (tmp.TerminatedOrResigned==0)
tmp.loc[:, 'target']=0
tmp.loc[index, 'target']=1
tmp.target.value_counts()

0    253639
1      4650
Name: target, dtype: int64

In [34]:
df.Timeline = pd.to_datetime(df.Timeline)

In [35]:
print(df.shape)
df = pd.merge(left=tmp, right=df, how = 'inner', on=['PIN','Timeline'])
print(df.shape)

(342200, 132)
(196785, 141)


In [36]:
df = df.sort_values(by=['Timeline','PIN'], ascending=False)

#### Save to csv

In [37]:
df.target.value_counts()

0    192990
1      3795
Name: target, dtype: int64

In [38]:
df.columns

Index(['PIN', 'Timeline', 'DateofTermination', 'City', 'Country',
       'Specialization', 'Sex', 'TerminatedOrResigned',
       'MonthsInCurrentPosition', 'TL_number',
       ...
       'English training_9', 'Number of courses_9', 'Projects_9',
       'SickLeave_9', 'Training days_9', 'Vacation_9', 'comments_9', 'likes_9',
       'pages_loaded_9', 'posts_9'],
      dtype='object', length=141)

In [39]:
## null check
for col in df.columns:
    if np.sum(df[col].isnull())>0: print(col, np.sum(df[col].isnull()))

DateofTermination 192990
TerminatedOrResigned 192990


In [40]:
path = 'data/data_'+suffix+'.csv'
print(path)
df.to_csv(path, sep=';', index=False)

data/data_Transform_data.csv
